In [1]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta
from joblib import dump, load

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

In [3]:
def get_margin_asset(name:str):
    """returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    return list(filter(lambda x: x['asset'] == name, client.get_margin_account()["userAssets"]))[0]

In [29]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '36.07116298',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '36.07116298'}

In [30]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00003288',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00003288'}

In [31]:
get_margin_asset("XMR")

{'asset': 'XMR',
 'free': '0.00000846',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00000846'}